In [3]:
import urllib3
http = urllib3.PoolManager()
r = http.request('GET',"https://raw.githubusercontent.com/dwyl/english-words/master/words.txt",preload_content=False)
with open("data/words.txt",'wb') as out:
    data = r.read()
    out.write (data)
r.release_conn()



/home/samyak/anaconda3/envs/py36/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [4]:
word_list = open('data/words.txt').readlines()
word_list = map(str.strip,word_list)

In [7]:
word_list = [word for word in word_list if len(word) == 3]
word_list = [word for word in word_list if word[0].islower()]
word_list = [word for word in word_list if word.isalpha()]
word_list = list(map(str.lower, word_list))
len(word_list)

1017

In [10]:
import numpy as np
word_list = np.asarray(word_list)
word_list.dtype
word_list.sort()

In [11]:
word_bytes = np.ndarray((word_list.size,word_list.itemsize),dtype = 'uint8', buffer = word_list.data)


In [12]:
word_bytes.shape
word_bytes = word_bytes[:, ::word_list.itemsize//3]
word_bytes.shape

(1017, 3)

In [14]:
word_list[0:4]
word_bytes[0:4]
from scipy.spatial.distance import pdist, squareform
from scipy.sparse import csr_matrix

hamming_dist = pdist(word_bytes, metric='hamming')
hamming_dist.shape

(516636,)

In [15]:
word_list[0:4]

array(['aah', 'aal', 'abb', 'abd'], dtype='<U3')

In [16]:
word_bytes[0:4]

array([[ 97,  97, 104],
       [ 97,  97, 108],
       [ 97,  98,  98],
       [ 97,  98, 100]], dtype=uint8)

In [17]:
squareform(hamming_dist)[0:4,0:4]

array([[0.        , 0.33333333, 0.66666667, 0.66666667],
       [0.33333333, 0.        , 0.66666667, 0.66666667],
       [0.66666667, 0.66666667, 0.        , 0.33333333],
       [0.66666667, 0.66666667, 0.33333333, 0.        ]])

In [18]:
mat = squareform(hamming_dist < 1.5/3)
mat.shape

(1017, 1017)

In [19]:
mat[0:4,0:4]

array([[False,  True, False, False],
       [ True, False, False, False],
       [False, False, False,  True],
       [False, False,  True, False]])

In [20]:
graph = csr_matrix(mat)

In [21]:
i1 = word_list.searchsorted('and')
i2 = word_list.searchsorted('dog')
word_list[i1]
word_list[i2]

'dog'

In [22]:
from scipy.sparse.csgraph import dijkstra
distances, predecessors = dijkstra(graph,indices=i1,return_predecessors=True)
print(distances[i2])

5.0


In [24]:
predecessors 

array([  64,   65,    3, ...,  935,  727, 1012], dtype=int32)

In [25]:
path = []
i = i2
while i != i1 :
    path.append(word_list[i])
    i = predecessors[i]
path.append(word_list[i1])
print(path[::-1])


['and', 'aud', 'mud', 'mug', 'dug', 'dog']


In [26]:
from scipy.sparse.csgraph import connected_components
N_components, component_list = connected_components(graph)
print(N_components)
print(component_list.shape)
component_list[0:5]

9
(1017,)


array([0, 0, 0, 0, 0], dtype=int32)

In [27]:
[np.sum(component_list == i) for i in range(N_components)]

[1009, 1, 1, 1, 1, 1, 1, 1, 1]

In [28]:
[list(word_list[np.where(component_list == i)]) for i in range(1,N_components)]

[['bcf'], ['bdl'], ['epi'], ['mmf'], ['sml'], ['wjc'], ['xcl'], ['xyz']]

In [29]:
distances,predecessors = dijkstra(graph, return_predecessors=True)
max_distance = np.max(distances[~np.isinf(distances)])
print(max_distance)

11.0


In [30]:
i1,i2 = np.where(distances == max_distance)
list(zip(word_list[i1],word_list[i2]))

[('dlr', 'hmm'),
 ('dlr', 'kmc'),
 ('hmm', 'dlr'),
 ('hmm', 'lbf'),
 ('hmm', 'rfz'),
 ('hmm', 'tfr'),
 ('kmc', 'dlr'),
 ('kmc', 'tfr'),
 ('lbf', 'hmm'),
 ('rfz', 'hmm'),
 ('tfr', 'hmm'),
 ('tfr', 'kmc')]

In [31]:
distances

array([[0., 1., 3., ..., 4., 4., 4.],
       [1., 0., 2., ..., 5., 5., 4.],
       [3., 2., 0., ..., 4., 4., 4.],
       ...,
       [4., 5., 4., ..., 0., 1., 3.],
       [4., 5., 4., ..., 1., 0., 4.],
       [4., 4., 4., ..., 3., 4., 0.]])

In [32]:
for j,i in zip(i1,i2):
    path = []
    while i!=j:
        path.append(word_list[i])
        i = predecessors[j,i]
    path.append(word_list[j])
    print(path[::-1])

['dlr', 'tlr', 'tlo', 'too', 'tot', 'tgt', 'ugt', 'ust', 'usu', 'umu', 'umm', 'hmm']
['dlr', 'tlr', 'tlo', 'blo', 'blk', 'ilk', 'ick', 'ich', 'iph', 'kph', 'kpc', 'kmc']
['hmm', 'umm', 'umu', 'usu', 'ush', 'bsh', 'boh', 'boo', 'blo', 'tlo', 'tlr', 'dlr']
['hmm', 'umm', 'umu', 'usu', 'ust', 'ast', 'abt', 'abl', 'dbl', 'dbw', 'lbw', 'lbf']
['hmm', 'umm', 'ump', 'unp', 'uns', 'uhs', 'chs', 'cfs', 'cfm', 'sfm', 'sfz', 'rfz']
['hmm', 'umm', 'umu', 'usu', 'ush', 'bsh', 'boh', 'boo', 'blo', 'tlo', 'tlr', 'tfr']
['kmc', 'kpc', 'kph', 'iph', 'ich', 'ick', 'ilk', 'blk', 'blo', 'tlo', 'tlr', 'dlr']
['kmc', 'kpc', 'kph', 'iph', 'ich', 'ick', 'ilk', 'blk', 'blo', 'tlo', 'tlr', 'tfr']
['lbf', 'lbw', 'dbw', 'daw', 'baw', 'bah', 'bsh', 'ush', 'usu', 'umu', 'umm', 'hmm']
['rfz', 'rfb', 'rub', 'rud', 'rnd', 'end', 'enc', 'unc', 'unp', 'ump', 'umm', 'hmm']
['tfr', 'tlr', 'tlo', 'too', 'tot', 'tgt', 'ugt', 'ust', 'usu', 'umu', 'umm', 'hmm']
['tfr', 'tlr', 'tlo', 'blo', 'blk', 'ilk', 'ick', 'ich', 'iph', '